# 🏦 Bank Statement Processor - Complete Python Analysis

## Project Configuration
| Resource | ID |
|----------|----|
| **Spreadsheet ID** | `1XuvPyWNhB3WAOMHDO9wXkZ5AO36Cce13PH8PAojG9Eo` |
| **Script ID** | `1Y40DccCVEpn29uA3P0gvQpyWmINnM_9CVZ7YzLqSQPieFGUBd3s83oa9` |
| **Input Folder ID** | `1vVQC5F8ZrKZnqv5QAWcye5VKGxCQ1c3q` |
| **Processed Folder ID** | `198Xe7BBn3ibRgXoUnOOz2NMWRcPgNtQj` |
| **Output Folder ID** | `1GQxr8YiFnm23YM78k77DYzT0tF1CIQMp` |

---

**Last Updated:** 2026-01-20

## 📦 CELL 1: Install Dependencies

In [ ]:
# ============================================================
# CELL 1: Install Required Packages
# ============================================================
!pip install -q gspread google-auth google-auth-oauthlib google-auth-httplib2
!pip install -q pandas numpy matplotlib seaborn
!pip install -q PyPDF2 pdfplumber tabula-py
!pip install -q google-api-python-client

print('✅ All packages installed successfully!')

## 🔐 CELL 2: Authentication & Configuration

In [ ]:
# ============================================================
# CELL 2: Authentication & Configuration
# ============================================================
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import pandas as pd
import numpy as np
import io
import os
from datetime import datetime

# Authorize
creds, project = default()
gc = gspread.authorize(creds)

# Build Drive service
drive_service = build('drive', 'v3', credentials=creds)

# ============================================================
# 🔧 CONFIGURATION - YOUR PROJECT IDS
# ============================================================
CONFIG = {
    'SPREADSHEET_ID': '1XuvPyWNhB3WAOMHDO9wXkZ5AO36Cce13PH8PAojG9Eo',
    'SCRIPT_ID': '1Y40DccCVEpn29uA3P0gvQpyWmINnM_9CVZ7YzLqSQPieFGUBd3s83oa9',
    'INPUT_FOLDER_ID': '1vVQC5F8ZrKZnqv5QAWcye5VKGxCQ1c3q',
    'PROCESSED_FOLDER_ID': '198Xe7BBn3ibRgXoUnOOz2NMWRcPgNtQj',
    'OUTPUT_FOLDER_ID': '1GQxr8YiFnm23YM78k77DYzT0tF1CIQMp',
    'ARCHIVE_FOLDER_ID': '1KQ31p9k1QSgh2dgteEWuDhrs0x3la-4n'
}

print('✅ Authentication successful!')
print(f'📊 Project: {project}')
print(f'📋 Spreadsheet ID: {CONFIG["SPREADSHEET_ID"]}')

## 📊 CELL 3: Connect to Spreadsheet

In [ ]:
# ============================================================
# CELL 3: Connect to Google Spreadsheet
# ============================================================
try:
    spreadsheet = gc.open_by_key(CONFIG['SPREADSHEET_ID'])
    print(f'✅ Connected to: {spreadsheet.title}')
    print(f'📊 Available Sheets:')
    for i, ws in enumerate(spreadsheet.worksheets()):
        print(f'   {i+1}. {ws.title} ({ws.row_count} rows)')
except Exception as e:
    print(f'❌ Connection Error: {e}')
    print('\n🔧 Troubleshooting:')
    print('   1. Make sure the spreadsheet is shared with your Google account')
    print('   2. Verify the SPREADSHEET_ID is correct')

## 📁 CELL 4: List PDFs in Input Folder

In [ ]:
# ============================================================
# CELL 4: List PDFs in Input Folder
# ============================================================
def list_pdfs_in_folder(folder_id):
    """List all PDF files in a Google Drive folder"""
    query = f"'{folder_id}' in parents and mimeType='application/pdf' and trashed=false"
    results = drive_service.files().list(
        q=query,
        fields='files(id, name, size, modifiedTime)',
        orderBy='modifiedTime desc'
    ).execute()
    return results.get('files', [])

# List PDFs
print('📁 Scanning Input Folder for PDFs...\n')
pdf_files = list_pdfs_in_folder(CONFIG['INPUT_FOLDER_ID'])

if pdf_files:
    print(f'✅ Found {len(pdf_files)} PDF file(s):\n')
    for i, f in enumerate(pdf_files):
        size_kb = int(f.get('size', 0)) / 1024
        print(f'   {i+1}. {f["name"]}')
        print(f'      ID: {f["id"]}')
        print(f'      Size: {size_kb:.1f} KB')
        print(f'      Modified: {f["modifiedTime"]}')
        print()
else:
    print('⚠️ No PDF files found in the input folder')
    print(f'   Folder ID: {CONFIG["INPUT_FOLDER_ID"]}')

## 📄 CELL 5: Download & Extract PDF Text

In [ ]:
# ============================================================
# CELL 5: Download & Extract PDF Text
# ============================================================
import pdfplumber
import re

def download_pdf(file_id, filename):
    """Download a PDF from Google Drive"""
    request = drive_service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while not done:
        status, done = downloader.next_chunk()
    fh.seek(0)
    
    # Save locally
    local_path = f'/content/{filename}'
    with open(local_path, 'wb') as f:
        f.write(fh.read())
    return local_path

def extract_text_from_pdf(pdf_path):
    """Extract text from PDF using pdfplumber"""
    text = ''
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + '\n'
    return text

# Process first PDF as example
if pdf_files:
    first_pdf = pdf_files[0]
    print(f'📄 Processing: {first_pdf["name"]}\n')
    
    # Download
    local_path = download_pdf(first_pdf['id'], first_pdf['name'])
    print(f'✅ Downloaded to: {local_path}')
    
    # Extract text
    extracted_text = extract_text_from_pdf(local_path)
    print(f'✅ Extracted {len(extracted_text)} characters\n')
    
    # Show preview
    print('📝 Text Preview (first 1000 chars):')
    print('=' * 50)
    print(extracted_text[:1000])
    print('=' * 50)
else:
    print('⚠️ No PDFs to process')

## 🏦 CELL 6: Parse Bank Statement

In [ ]:
# ============================================================
# CELL 6: Parse Bank Statement Transactions
# ============================================================
import uuid

def detect_bank_type(text):
    """Detect the bank type from statement text"""
    lower_text = text.lower()
    if 'capital one' in lower_text or '360 checking' in lower_text:
        return 'CAPITAL_ONE'
    elif 'chase' in lower_text or 'jpmorgan' in lower_text:
        return 'CHASE'
    elif 'bank of america' in lower_text:
        return 'BANK_OF_AMERICA'
    elif 'wells fargo' in lower_text:
        return 'WELLS_FARGO'
    return 'UNKNOWN'

def parse_capital_one(text, source_file):
    """Parse Capital One bank statement"""
    transactions = []
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    
    # Transaction pattern: "Sep 3 Description Credit +$XX.XX $XX.XX"
    pattern = r'^(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+(\d{1,2})\s+(.+?)\s+([+-]?\$?[\d,]+\.\d{2})\s+\$?([\d,]+\.\d{2})$'
    
    months = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6,
              'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}
    
    for line in lines:
        if 'Opening Balance' in line or 'Closing Balance' in line:
            continue
        
        match = re.match(pattern, line, re.IGNORECASE)
        if match:
            month, day, description, amount_str, balance_str = match.groups()
            
            # Parse amount
            amount = float(re.sub(r'[$,\s+]', '', amount_str))
            if '-' in amount_str or 'Debit' in line:
                amount = -abs(amount)
            
            # Create date
            year = datetime.now().year
            trans_date = datetime(year, months[month], int(day))
            
            transactions.append({
                'id': str(uuid.uuid4())[:8],
                'date': trans_date.strftime('%Y-%m-%d'),
                'description': description.strip(),
                'category': '',
                'amount': amount,
                'type': 'Income' if amount >= 0 else 'Expense',
                'balance': float(re.sub(r'[$,]', '', balance_str)),
                'bank': 'Capital One',
                'source_file': source_file
            })
    
    return transactions

def parse_generic(text, source_file):
    """Generic parser for unknown bank formats"""
    transactions = []
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    
    # Generic pattern: date + description + amount
    pattern = r'(\d{1,2}[/\-]\d{1,2}[/\-]?\d{0,4})\s+(.+?)\s+([+-]?\$?[\d,]+\.\d{2})$'
    
    for line in lines:
        match = re.search(pattern, line)
        if match:
            date_str, description, amount_str = match.groups()
            
            amount = float(re.sub(r'[$,\s+]', '', amount_str))
            if '-' in amount_str:
                amount = -abs(amount)
            
            transactions.append({
                'id': str(uuid.uuid4())[:8],
                'date': date_str,
                'description': description.strip(),
                'category': '',
                'amount': amount,
                'type': 'Income' if amount >= 0 else 'Expense',
                'balance': None,
                'bank': 'Unknown',
                'source_file': source_file
            })
    
    return transactions

# Parse the extracted text
if 'extracted_text' in dir() and extracted_text:
    bank_type = detect_bank_type(extracted_text)
    print(f'🏦 Detected Bank: {bank_type}\n')
    
    if bank_type == 'CAPITAL_ONE':
        transactions = parse_capital_one(extracted_text, first_pdf['name'])
    else:
        transactions = parse_generic(extracted_text, first_pdf['name'])
    
    print(f'✅ Parsed {len(transactions)} transactions\n')
    
    if transactions:
        df_transactions = pd.DataFrame(transactions)
        display(df_transactions)
else:
    print('⚠️ No extracted text available. Run Cell 5 first.')

## 💾 CELL 7: Write Transactions to Sheet

In [ ]:
# ============================================================
# CELL 7: Write Transactions to Google Sheet
# ============================================================
def write_transactions_to_sheet(spreadsheet, transactions, sheet_name='Transactions'):
    """Write transactions to Google Sheet"""
    try:
        # Get or create sheet
        try:
            sheet = spreadsheet.worksheet(sheet_name)
        except:
            sheet = spreadsheet.add_worksheet(title=sheet_name, rows=1000, cols=12)
            # Add headers
            headers = ['ID', 'Date', 'Description', 'Category', 'Amount', 'Type', 
                       'Balance', 'Bank', 'Account', 'Period', 'Processed', 'Source']
            sheet.update('A1:L1', [headers])
        
        # Convert transactions to rows
        rows = []
        for t in transactions:
            rows.append([
                t.get('id', ''),
                t.get('date', ''),
                t.get('description', ''),
                t.get('category', ''),
                t.get('amount', 0),
                t.get('type', ''),
                t.get('balance', ''),
                t.get('bank', ''),
                t.get('account', ''),
                t.get('period', ''),
                datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                t.get('source_file', '')
            ])
        
        # Append to sheet
        if rows:
            next_row = len(sheet.get_all_values()) + 1
            sheet.update(f'A{next_row}:L{next_row + len(rows) - 1}', rows)
            return len(rows)
        return 0
        
    except Exception as e:
        print(f'❌ Error writing to sheet: {e}')
        return 0

# Write transactions
if 'transactions' in dir() and transactions:
    count = write_transactions_to_sheet(spreadsheet, transactions)
    print(f'✅ Written {count} transactions to sheet!')
    print(f'📊 View spreadsheet: https://docs.google.com/spreadsheets/d/{CONFIG["SPREADSHEET_ID"]}')
else:
    print('⚠️ No transactions to write. Run Cell 6 first.')

## 📦 CELL 8: Move PDF to Processed Folder

In [ ]:
# ============================================================
# CELL 8: Move PDF to Processed Folder
# ============================================================
def move_file_to_folder(file_id, source_folder_id, dest_folder_id):
    """Move a file from one folder to another in Google Drive"""
    try:
        # Get current parents
        file = drive_service.files().get(
            fileId=file_id,
            fields='parents'
        ).execute()
        
        previous_parents = ','.join(file.get('parents', []))
        
        # Move to new folder
        drive_service.files().update(
            fileId=file_id,
            addParents=dest_folder_id,
            removeParents=previous_parents,
            fields='id, parents'
        ).execute()
        
        return True
    except Exception as e:
        print(f'❌ Error moving file: {e}')
        return False

# Move the processed PDF
if 'first_pdf' in dir() and first_pdf:
    print(f'📦 Moving {first_pdf["name"]} to Processed folder...\n')
    
    success = move_file_to_folder(
        first_pdf['id'],
        CONFIG['INPUT_FOLDER_ID'],
        CONFIG['PROCESSED_FOLDER_ID']
    )
    
    if success:
        print(f'✅ Successfully moved to Processed folder!')
        print(f'📂 Processed Folder: https://drive.google.com/drive/folders/{CONFIG["PROCESSED_FOLDER_ID"]}')
    else:
        print('❌ Failed to move file')
else:
    print('⚠️ No PDF to move. Run previous cells first.')

## 🔄 CELL 9: Process ALL PDFs (Batch)

In [ ]:
# ============================================================
# CELL 9: Process ALL PDFs in Batch
# ============================================================
def process_all_pdfs():
    """Process all PDFs in the input folder"""
    print('🚀 Starting batch processing...\n')
    
    # Get all PDFs
    pdf_files = list_pdfs_in_folder(CONFIG['INPUT_FOLDER_ID'])
    
    if not pdf_files:
        print('⚠️ No PDFs found in input folder')
        return
    
    print(f'📁 Found {len(pdf_files)} PDF(s) to process\n')
    
    total_transactions = 0
    processed_count = 0
    error_count = 0
    
    for pdf in pdf_files:
        print(f'\n📄 Processing: {pdf["name"]}')
        print('=' * 50)
        
        try:
            # Download
            local_path = download_pdf(pdf['id'], pdf['name'])
            print(f'   ✅ Downloaded')
            
            # Extract text
            text = extract_text_from_pdf(local_path)
            print(f'   ✅ Extracted {len(text)} chars')
            
            # Detect bank and parse
            bank_type = detect_bank_type(text)
            print(f'   🏦 Bank: {bank_type}')
            
            if bank_type == 'CAPITAL_ONE':
                transactions = parse_capital_one(text, pdf['name'])
            else:
                transactions = parse_generic(text, pdf['name'])
            
            print(f'   ✅ Parsed {len(transactions)} transactions')
            
            # Write to sheet
            if transactions:
                count = write_transactions_to_sheet(spreadsheet, transactions)
                total_transactions += count
                print(f'   ✅ Written to sheet')
            
            # Move to processed
            move_file_to_folder(pdf['id'], CONFIG['INPUT_FOLDER_ID'], CONFIG['PROCESSED_FOLDER_ID'])
            print(f'   ✅ Moved to Processed folder')
            
            processed_count += 1
            
            # Cleanup local file
            os.remove(local_path)
            
        except Exception as e:
            print(f'   ❌ Error: {e}')
            error_count += 1
    
    # Summary
    print('\n' + '=' * 50)
    print('📊 BATCH PROCESSING COMPLETE')
    print('=' * 50)
    print(f'   ✅ Processed: {processed_count} files')
    print(f'   📝 Transactions: {total_transactions}')
    print(f'   ❌ Errors: {error_count}')
    print(f'\n📊 View results: https://docs.google.com/spreadsheets/d/{CONFIG["SPREADSHEET_ID"]}')

# Run batch processing
process_all_pdfs()

## 📊 CELL 10: Load & Analyze Transactions

In [ ]:
# ============================================================
# CELL 10: Load & Analyze Transactions from Sheet
# ============================================================
try:
    transactions_sheet = spreadsheet.worksheet('Transactions')
    data = transactions_sheet.get_all_values()
    
    if len(data) > 1:
        df = pd.DataFrame(data[1:], columns=data[0])
        print(f'✅ Loaded {len(df)} transactions\n')
        
        # Convert Amount to numeric
        df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')
        
        # Summary statistics
        total_income = df[df['Amount'] > 0]['Amount'].sum()
        total_expenses = df[df['Amount'] < 0]['Amount'].sum()
        net_flow = total_income + total_expenses
        
        print('📊 FINANCIAL SUMMARY')
        print('=' * 40)
        print(f'💰 Total Income:    ${total_income:,.2f}')
        print(f'💸 Total Expenses:  ${abs(total_expenses):,.2f}')
        print(f'📈 Net Cash Flow:   ${net_flow:,.2f}')
        print('=' * 40)
        
        # Show recent transactions
        print('\n📋 Recent Transactions:')
        display(df.tail(10))
    else:
        print('⚠️ No transaction data found in sheet')
        
except Exception as e:
    print(f'❌ Error loading transactions: {e}')

## 📈 CELL 11: Visualizations

In [ ]:
# ============================================================
# CELL 11: Visualizations
# ============================================================
import matplotlib.pyplot as plt
import seaborn as sns

if 'df' in dir() and len(df) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('💰 Bank Statement Analysis', fontsize=16, fontweight='bold')
    
    # 1. Income vs Expenses
    ax1 = axes[0, 0]
    income_expense = [total_income, abs(total_expenses)]
    colors = ['#2ecc71', '#e74c3c']
    ax1.bar(['Income', 'Expenses'], income_expense, color=colors)
    ax1.set_title('Income vs Expenses')
    ax1.set_ylabel('Amount ($)')
    
    # 2. Transaction type distribution
    ax2 = axes[0, 1]
    type_counts = df['Type'].value_counts()
    ax2.pie(type_counts, labels=type_counts.index, autopct='%1.1f%%', colors=['#2ecc71', '#e74c3c'])
    ax2.set_title('Transaction Types')
    
    # 3. Amount distribution
    ax3 = axes[1, 0]
    df['Amount'].hist(bins=20, ax=ax3, color='#3498db', edgecolor='white')
    ax3.set_title('Transaction Amount Distribution')
    ax3.set_xlabel('Amount ($)')
    ax3.set_ylabel('Frequency')
    
    # 4. Cumulative balance (if available)
    ax4 = axes[1, 1]
    if 'Balance' in df.columns:
        balance = pd.to_numeric(df['Balance'], errors='coerce').dropna()
        if len(balance) > 0:
            balance.plot(ax=ax4, color='#9b59b6', linewidth=2)
            ax4.set_title('Balance Over Time')
            ax4.set_ylabel('Balance ($)')
    else:
        # Running total
        running_total = df['Amount'].cumsum()
        running_total.plot(ax=ax4, color='#9b59b6', linewidth=2)
        ax4.set_title('Cumulative Cash Flow')
        ax4.set_ylabel('Amount ($)')
    
    plt.tight_layout()
    plt.show()
else:
    print('⚠️ No data to visualize. Run Cell 10 first.')